- выбрала вариант 2: собрать из открытых источников что-то похожее на датасет предсказания банкротства тайваньских компаний.

- простая часть - скачала основные xml-файлы с https://www.nalog.gov.ru/opendata/ и несколько справочников csv и xlsx - около 30 гб

- с информацией с сайта https://bo.nalog.ru/ чуть сложнее - ссылка на страницу с отчётами осуществляется по id карточки организации, который можно узнать, сделав запрос по номеру инн юр. лица (однозначно определяет организацию)

- далее, отчёты есть только для юридических лиц (т.е. необходимо знать ИННЮЛ), тех из них, которые обязаны предоставлять открытую информацию (ИНН физ. лиц не нужны)

- т.е. необходим список иннюл - его можно достать из реестра МСП (малых и средних предприятий), чтобы первым делом начать собирать ту информацию, к которой нет прямого доступа.

- xml достаточно громоздкие, но к ним есть описание структуры и данные достаточно чистые (есть немного непечатаемых символов), основная трудность возникла с кодировкой.

In [9]:
import os
import pickle
import requests

from lxml import etree

In [5]:
# путь к директории с файлами реестра мсп, около 20 гб, около 7 тыс. файлов
path = '../raw_data/reestr_ms/data-10032023-structure-10032022'
files = [path + '/' + file for file in os.listdir(path)]
len(files)

6794

In [8]:
#  пусть будет 7 отрезков по 1000 файлов (в последнем - 794)
k = 1000
cut_files = [files[i: i + k] for i in range(0, 7000, 1000) ]

- получилось 7 дампов pickle, примерно по 350 тыс ИННЮЛ в каждом списке
- общая длина - около 2.3 млн
- здесь обработаем и сразу запишем для удобства в несколько файлов (оказалось, что для паука можно было делать файлы ещё короче)

  Wall time: 15min 44s

In [ ]:
# %%time

# for j, files in enumerate(cut_files):
    
#     inn = []
#     for i, file in enumerate(files):
        
#         if i % 100 == 0: print(i)
        
#         tree = etree.parse(file)
#         root = tree.getroot()
#         
#         for doc in root.findall('Документ/ОргВклМСП'):
#             inn.append(doc.attrib['ИННЮЛ'])
            
    
#     with open(f'parsed_data/inn/inn_{j}.pickle', 'wb') as f:
#         pickle.dump(inn, f)  

- дальше более подробно изучила, какая информация есть на страничках с отчётами разных организаций, для примера:

    https://bo.nalog.ru/organizations-card/5581428 - ООО, бух баланс + отчёт о фин.рез - упрощённая форма

    https://bo.nalog.ru/organizations-card/5573882 - ООО, бух баланс и фин рез - видимо, обычная форма

    https://bo.nalog.ru/organizations-card/6838444 - ОАО, бух баланс, фин рез, изм капитала, движение денежн средств, аудит заключение, пояснения...

    https://bo.nalog.ru/organizations-card/5581537 - производств кооператив, бух бал, фин рез

    https://bo.nalog.ru/organizations-card/5576949 - АО, бух бал, фин рез

    https://bo.nalog.ru/organizations-card/2258337 - ОООб бух бал, фин рез, отчёт о целев исп денежн срв, отчёт об изм капит, отчёт о движ денежн срв


- бухгалтерский баланс за 3 года и финансовые результаты за 2 года есть для всех типов

- если не искать информацию целенаправленно по какому-то критерию (типу организации), собственности и т.п. - то можно собрать именно их

- но оказалось, что страницы генерируются динамически - простой запрос практически ничего не возвращает

In [10]:
url = 'https://bo.nalog.ru/search'
inn = '4702015870'
headers = {'User-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'}
params= {'query': inn, 'page':0}

In [11]:
req = requests.get(url, headers=headers, params=params)
req.d
req.text

'<!doctype html><html xmlns="http://www.w3.org/1999/xhtml" lang="ru"><head><meta charset="utf-8"><meta content="" name="Description"><meta content="" name="Keywords"><meta http-equiv="X-UA-Compatible" content="IE=edge"><meta name="viewport" content="width=device-width,initial-scale=1"><link rel="shortcut icon" type="image/jpg" href="favicon.ico"/><title>Ð\xa0ÐµÑ\x81Ñ\x83Ñ\x80Ñ\x81 Ð\x91Ð¤Ð\x9e</title><link href="/static/css/main.a64ce339.css" rel="stylesheet"></head><body><div id="root" class="root"></div><div id="modal"></div><div id="side"></div><script type="text/javascript" src="/static/js/main.abd0c8f2.js"></script></body></html>'

- для загрузки страницы отправляется множество get-запросов, каждый из которых возвращает json для наполнения контентом.
- при этом, только для сбора основной информации (карточка компании, фин. результаты и бух. отчет) нужно минимум 4 запроса, при этом для остальных отчётов и части данных финансового результата у компаний с более сложным формами отчётности - нужны дополнительные запросы.

- итоговая логика переходов:

##### 1. карточка организации по инн: 

        f'https://bo.nalog.ru/nbo/organizations/search?query={ИНН}&page=0'
    
    - пустой ли json? response.json()
    - берём
        
        content: 0: id = id_org
                    
поле для okofs (форма собственности) - здесь пустой, для него отдельная вкладка


##### 2. переход на следующую страницу - по полученному id_org

        f'https://bo.nalog.ru/nbo/organizations/{id_org}'

    - берём
        
        shortName
        
        ogrn
        
        index
        
        region
        
        district
        
        city
        
        settlement - посёлок
        
        okved - не знаю, что, мб старая версия okvd
        
        okved2: 
            id - это версия okvd 2 отсюда https://www.consultant.ru/document/cons_doc_LAW_163320/c95bc15ad12ca2426a690bed4ff0b8686fbe62e4/
        
        okopf: 
            
            id - форма организации, см справочник okopf.csv  
        
        statusCode
        
        fullname: 
            
            registrationDate - дата регистрации, в едином реестре есть поле дата включения в МСП - и это разные даты
        
        authorizedCapital - уставной капитал
   
   
##### 3. снова переход по id_org

        f'https://bo.nalog.ru/nbo/organizations/{id_org}/bfo/'
        
    - чтобы получить id периода (по годам: для 2021 года данные за 2021, 2020, 2019)
    
        структура примерно такая:
            
            [{period: "2021", id: 12334, ...}, ...]
        
            забираем id=id_doc
            и на всякий случай формируем словарик по доступным периодам и их id
            
    
    
##### 4. забираем фин рез и бух баланс - по id_doc

        f'https://bo.nalog.ru/nbo/bfo/{id_doc}/details'
        
        
    - берём
        
        0:
            
            balance:
                okud - форма отчётности, см справочники для расшифровки столбцов
                и далее всё подряд, кроме столбцов expl
                
            financialResult:
                okud
                и всё подряд
                

In [12]:
headers = {'User-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'}
# реальный
inn = '3447001553'
# отсутствующий
inn1 = '12345678'

In [13]:
# в пауке понадобится конструктор, чтобы принять параметры
# и переопределить функцию start_requests чтобы итерироваться по списку инн
start_url = f'https://bo.nalog.ru/nbo/organizations/search?query={inn}&page=0'
response = requests.get(start_url, headers=headers)
data = response.json()

In [14]:
# получили id карточки организации - если не пустая - 
if data['content']:
    id_org = data['content'][0]['id']

    # здесь yeld с вызовом следующего метода, иначе ничего не возвращаем
id_org

5581428

In [15]:
# собираем основную информацию из карточки организации
# и составляем словарь, который передадим следующему методу
url_org_card = f'https://bo.nalog.ru/nbo/organizations/{id_org}'
response = requests.get(url_org_card, headers=headers)
data = response.json()

In [16]:
org_info = {}
# явно перечисляем ключи, так как проще видеть, что забираем 
# и легче выключить/включить какие-то данные
# в качестве _id - инн, так как по нему стыкуем с xml
org_info['_id'] = data['inn']
org_info['id_org'] = data['id']
org_info['shortName'] = data['shortName']
org_info['ogrn'] = data['ogrn']
org_info['index'] = data['index']
org_info['region'] = data['region']
org_info['district'] = data['district']
org_info['city'] = data['city']
org_info['settlement'] = data['settlement']
org_info['okved'] = data['okved']  # видимо, старая версия?
org_info['okved2'] = data.get('okved2').get('id')  # общерос классиф видов эк. деят-ти
org_info['okopf'] = data['okopf']['id']  # форма организации, см справочник okopf.csv
org_info['statusCode'] = data['statusCode']
org_info['registrationDate'] = data['registrationDate']
org_info['authorizedCapital'] = data['authorizedCapital']

In [18]:
# далее - страничка с отчётами, какие есть периоды
url_report_id = f'https://bo.nalog.ru/nbo/organizations/{id_org}/bfo/'
response = requests.get(url_report_id, headers=headers)
data = response.json()
data

[{'period': '2021',
  'publication': 200006,
  'actualBfoDate': '2022-03-29',
  'gainSum': None,
  'knd': None,
  'azFlg': False,
  'actualCorrectionNumber': None,
  'actualCorrectionDate': None,
  'publishedCorrectionNumber': None,
  'publishedCorrectionDate': None,
  'actives': 1396.0,
  'isCb': None,
  'id': 14606812,
  'clarification': None,
  'auditReport': None,
  'published': False},
 {'period': '2020',
  'publication': 202005,
  'actualBfoDate': '2021-04-02',
  'gainSum': None,
  'knd': None,
  'azFlg': False,
  'actualCorrectionNumber': None,
  'actualCorrectionDate': None,
  'publishedCorrectionNumber': None,
  'publishedCorrectionDate': None,
  'actives': 664.0,
  'isCb': None,
  'id': 7721040,
  'clarification': None,
  'auditReport': None,
  'published': False},
 {'period': '2019',
  'publication': 202005,
  'actualBfoDate': '2020-04-06',
  'gainSum': None,
  'knd': None,
  'azFlg': False,
  'actualCorrectionNumber': None,
  'actualCorrectionDate': None,
  'publishedCorrec

In [19]:
# проверяем, есть ли нужный период и одновременно
# итерируемся по списку, чтобы собрать словарик {period: id}
report_periods = {reports_card['period']: reports_card['id'] for reports_card in data}
report_periods

{'2021': 14606812, '2020': 7721040, '2019': 3056434}

In [21]:
# период пойдёт в конструктор паука
# для небольшой части компаний уже есть отчёт за 2022, но в основном - только 2021
year = '2021'
if year in report_periods:    
    id_period = report_periods[year]

    # добавляем нераспакованный словарик с периодами в общую информацию - вдруг пригодится
    # и передаём словарь дальше
    org_info['report_periods'] = report_periods
    # yeld к следующей функции по сконструированному url_reports

id_period

14606812

In [22]:
# страничка с отчётами
url_reports = f'https://bo.nalog.ru/nbo/bfo/{id_period}/details'
response = requests.get(url_reports, headers=headers)
data = response.json()[0]

In [23]:
# далее - опять явно указываю столбцы, чтобы контролировать, что забираю - и включать/выключать
# их много
# смешиваю balance и fin_result -  так как коды не пересекаются
balance = data['balance']
fin_res = data['financialResult']

In [24]:
# здесь для упрощённой формы и обычной - присутствуют все ключи
# но большая часть полей будет None
if balance['okud'] == '0710001':
    # АКТИВ
    # I Внеоборотные активы
    # Нематериальные активы
    org_info[f'{year}_1110'] = balance.get('current1110')
    org_info[f'{int(year) - 1}_1110'] = balance.get('previous1110')
    org_info[f'{int(year) - 2}_1110'] = balance.get('beforePrevious1110')
    # Результаты исследований и разработок
    org_info[f'{year}_1120'] = balance.get('current1120')
    org_info[f'{int(year) - 1}_1120'] = balance.get('previous1120')
    org_info[f'{int(year) - 2}_1120'] = balance.get('beforePrevious1120')
    # Нематериальные поисковые активы
    org_info[f'{year}_1130'] = balance.get('current1130')
    org_info[f'{int(year) - 1}_1130'] = balance.get('previous1130')
    org_info[f'{int(year) - 2}_1130'] = balance.get('beforePrevious1130')
    # Материальные поисковые активы
    org_info[f'{year}_1140'] = balance.get('current1140')
    org_info[f'{int(year) - 1}_1140'] = balance.get('previous1140')
    org_info[f'{int(year) - 2}_1140'] = balance.get('beforePrevious1140')
    # Основные средства
    org_info[f'{year}_1150'] = balance.get('current1150')
    org_info[f'{int(year) - 1}_1150'] = balance.get('previous1150')
    org_info[f'{int(year) - 2}_1150'] = balance.get('beforePrevious1150')
    # Доходные вложения в материальные ценности
    org_info[f'{year}_1160'] = balance.get('current1160')
    org_info[f'{int(year) - 1}_1160'] = balance.get('previous1160')
    org_info[f'{int(year) - 2}_1160'] = balance.get('beforePrevious1160')
    # Финансовые вложения
    org_info[f'{year}_1170'] = balance.get('current1170')
    org_info[f'{int(year) - 1}_1170'] = balance.get('previous1170')
    org_info[f'{int(year) - 2}_1170'] = balance.get('beforePrevious1170')
    # Отложенные налоговые активы
    org_info[f'{year}_1180'] = balance.get('current1180')
    org_info[f'{int(year) - 1}_1180'] = balance.get('previous1180')
    org_info[f'{int(year) - 2}_1180'] = balance.get('beforePrevious1180')
    # Прочие внеоборотные активы
    org_info[f'{year}_1190'] = balance.get('current1190')
    org_info[f'{int(year) - 1}_1190'] = balance.get('previous1190')
    org_info[f'{int(year) - 2}_1190'] = balance.get('beforePrevious1190')
    # Итого по разделу I
    org_info[f'{year}_1100'] = balance.get('current1100')
    org_info[f'{int(year) - 1}_1100'] = balance.get('previous1100')
    org_info[f'{int(year) - 2}_1100'] = balance.get('beforePrevious1100')
    # II Оборотные активы
    # Запасы
    org_info[f'{year}_1210'] = balance.get('current1210')
    org_info[f'{int(year) - 1}_1210'] = balance.get('previous1210')
    org_info[f'{int(year) - 2}_1210'] = balance.get('beforePrevious1210')
    # Налог на добавленную стоимость по приобретенным ценностям
    org_info[f'{year}_1120'] = balance.get('current1220')
    org_info[f'{int(year) - 1}_1220'] = balance.get('previous1220')
    org_info[f'{int(year) - 2}_1220'] = balance.get('beforePrevious1220')
    # Дебиторская задолженность
    org_info[f'{year}_1230'] = balance.get('current1230')
    org_info[f'{int(year) - 1}_1230'] = balance.get('previous1230')
    org_info[f'{int(year) - 2}_1230'] = balance.get('beforePrevious1230')
    # Финансовые вложения (за исключением денежных эквивалентов)
    org_info[f'{year}_1240'] = balance.get('current1240')
    org_info[f'{int(year) - 1}_1240'] = balance.get('previous1240')
    org_info[f'{int(year) - 2}_1240'] = balance.get('beforePrevious1240')
    # Денежные средства и денежные эквиваленты
    org_info[f'{year}_1250'] = balance.get('current1250')
    org_info[f'{int(year) - 1}_1250'] = balance.get('previous1250')
    org_info[f'{int(year) - 2}_1250'] = balance.get('beforePrevious1250')
    # Прочие оборотные активы
    org_info[f'{year}_1260'] = balance.get('current1260')
    org_info[f'{int(year) - 1}_1260'] = balance.get('previous1260')
    org_info[f'{int(year) - 2}_1260'] = balance.get('beforePrevious1260')
    # Итого по разделу II
    org_info[f'{year}_1200'] = balance.get('current1200')
    org_info[f'{int(year) - 1}_1200'] = balance.get('previous1200')
    org_info[f'{int(year) - 2}_1200'] = balance.get('beforePrevious1200')
    # Баланс по АКТИВЫ
    org_info[f'{year}_1600'] = balance.get('current1600')
    org_info[f'{int(year) - 1}_1600'] = balance.get('previous1600')
    org_info[f'{int(year) - 2}_1600'] = balance.get('beforePrevious1600')
    # ПАССИВ
    # III Капитал и резервы
    # Уставный капитал (складочный капитал, уставный фонд, вклады товарищей)
    org_info[f'{year}_1310'] = balance.get('current1310')
    org_info[f'{int(year) - 1}_1310'] = balance.get('previous1310')
    org_info[f'{int(year) - 2}_1310'] = balance.get('beforePrevious1310')
    # Собственные акции, выкупленные у акционеров
    org_info[f'{year}_1320'] = balance.get('current1320')
    org_info[f'{int(year) - 1}_1320'] = balance.get('previous1320')
    org_info[f'{int(year) - 2}_1320'] = balance.get('beforePrevious1320')
    # Переоценка внеоборотных активов
    org_info[f'{year}_1340'] = balance.get('current1340')
    org_info[f'{int(year) - 1}_1340'] = balance.get('previous1340')
    org_info[f'{int(year) - 2}_1340'] = balance.get('beforePrevious1340')
    # Добавочный капитал (без переоценки)
    org_info[f'{year}_1350'] = balance.get('current1350')
    org_info[f'{int(year) - 1}_1350'] = balance.get('previous1350')
    org_info[f'{int(year) - 2}_1350'] = balance.get('beforePrevious1350')
    # Резервный капитал
    org_info[f'{year}_1360'] = balance.get('current1360')
    org_info[f'{int(year) - 1}_1360'] = balance.get('previous1360')
    org_info[f'{int(year) - 2}_1360'] = balance.get('beforePrevious1360')
    # Нераспределенная прибыль (непокрытый убыток)
    org_info[f'{year}_1370'] = balance.get('current1370')
    org_info[f'{int(year) - 1}_1370'] = balance.get('previous1370')
    org_info[f'{int(year) - 2}_1370'] = balance.get('beforePrevious1370')
    # Итого по разделу III
    org_info[f'{year}_1300'] = balance.get('current1300')
    org_info[f'{int(year) - 1}_1300'] = balance.get('previous1300')
    org_info[f'{int(year) - 2}_1300'] = balance.get('beforePrevious1300')
    # IV. Долгосрочные обязательства
    # Заемные средства
    org_info[f'{year}_1510'] = balance.get('current1510')
    org_info[f'{int(year) - 1}_1510'] = balance.get('previous1510')
    org_info[f'{int(year) - 2}_1510'] = balance.get('beforePrevious1510')
    # Кредиторская задолженность
    org_info[f'{year}_1520'] = balance.get('current1520')
    org_info[f'{int(year) - 1}_1520'] = balance.get('previous1520')
    org_info[f'{int(year) - 2}_1520'] = balance.get('beforePrevious1520')
    # Доходы будущих периодов
    org_info[f'{year}_1530'] = balance.get('current1530')
    org_info[f'{int(year) - 1}_1530'] = balance.get('previous1530')
    org_info[f'{int(year) - 2}_1530'] = balance.get('beforePrevious1530')
    # Оценочные обязательства
    org_info[f'{year}_1540'] = balance.get('current1540')
    org_info[f'{int(year) - 1}_1540'] = balance.get('previous1540')
    org_info[f'{int(year) - 2}_1540'] = balance.get('beforePrevious1540')
    # Прочие обязательства
    org_info[f'{year}_1550'] = balance.get('current1550')
    org_info[f'{int(year) - 1}_1550'] = balance.get('previous1550')
    org_info[f'{int(year) - 2}_1550'] = balance.get('beforePrevious1550')
    # Итого по разделу V
    org_info[f'{year}_1500'] = balance.get('current1500')
    org_info[f'{int(year) - 1}_1500'] = balance.get('previous1500')
    org_info[f'{int(year) - 2}_1500'] = balance.get('beforePrevious1500')
    # Баланс по ПАССИВЫ
    org_info[f'{year}_1700'] = balance.get('current1700')
    org_info[f'{int(year) - 1}_1700'] = balance.get('previous1700')
    org_info[f'{int(year) - 2}_1700'] = balance.get('beforePrevious1700')  

In [25]:
# а здесь - для упрощённой формы не все ключи представлены
# плюс, кажется, данные подтягиваются из нескольких мест и не все публикуются - есть ещё id
# и get-запросы
# здесь - достаточно полный список
if fin_res['okud'] == '0710002':
    # Выручка
    org_info[f'{year}_2110'] = fin_res.get('current2110')
    org_info[f'{int(year) - 1}_2110'] = fin_res.get('previous2110')
    # Себестоимость продаж
    org_info[f'{year}_2120'] = fin_res.get('current2120')
    org_info[f'{int(year) - 1}_2120'] = fin_res.get('previous2120')
    # Валовая прибыль (убыток)
    org_info[f'{year}_2100'] = fin_res.get('current2100')
    org_info[f'{int(year) - 1}_2100'] = fin_res.get('previous2100')
    # Управленческие расходы
    org_info[f'{year}_2220'] = fin_res.get('current2220')
    org_info[f'{int(year) - 1}_2220'] = fin_res.get('previous2220')
    # Прибыль (убыток) от продаж
    org_info[f'{year}_2200'] = fin_res.get('current2200')
    org_info[f'{int(year) - 1}_2200'] = fin_res.get('previous2200')
    # Проценты к уплате
    org_info[f'{year}_2330'] = fin_res.get('current2330')
    org_info[f'{int(year) - 1}_2330'] = fin_res.get('previous2330')
    # Прочие доходы
    org_info[f'{year}_2340'] = fin_res.get('current2340')
    org_info[f'{int(year) - 1}_2340'] = fin_res.get('previous2340')
    # Прочие расходы
    org_info[f'{year}_2350'] = fin_res.get('current2350')
    org_info[f'{int(year) - 1}_2350'] = fin_res.get('previous2350')
    # Прибыль (убыток) до налогообложения
    org_info[f'{year}_2300'] = fin_res.get('current2300')
    org_info[f'{int(year) - 1}_2300'] = fin_res.get('previous2300')
    # Налог на прибыль
    org_info[f'{year}_2410'] = fin_res.get('current2410')
    org_info[f'{int(year) - 1}_2410'] = fin_res.get('previous2410')
#     # в т.ч. текущий налог на прибыль
#     org_info[f'{year}_2411'] = fin_res.get('current2411')
#     org_info[f'{int(year) - 1}_2411'] = fin_res.get('previous2411')
#     # Отложенный налог на прибыль
#     org_info[f'{year}_2412'] = fin_res.get('current2412')
#     org_info[f'{int(year) - 1}_2412'] = fin_res.get('previous2412')
    # Чистая прибыль (убыток) - есть вроде для всех
    org_info[f'{year}_2400'] = fin_res.get('current2400')
    org_info[f'{int(year) - 1}_2400'] = fin_res.get('previous2400')
    # Совокупный финансовый результат периода (не для упрощённой формы)
    org_info[f'{year}_2500'] = fin_res.get('current2500')
    org_info[f'{int(year) - 1}_2500'] = fin_res.get('previous2500')
    
    org_info[f'{year}_2110'] = fin_res.get('current2110')
    org_info[f'{int(year) - 1}_2110'] = fin_res.get('previous2110')
    
    org_info[f'{year}_2110'] = fin_res.get('current2110')
    org_info[f'{int(year) - 1}_2110'] = fin_res.get('previous2110')

In [26]:
# и уже этот документ идёт в Item
org_info

{'_id': '3447001553',
 'id_org': 5581428,
 'shortName': 'ООО "МОДЕРН-БОН"',
 'ogrn': '1023404293505',
 'index': '400066',
 'region': 'ВОЛГОГРАДСКАЯ',
 'district': None,
 'city': 'ВОЛГОГРАД',
 'settlement': None,
 'okved': None,
 'okved2': '93.19',
 'okopf': 12300,
 'statusCode': 'ACTIVE',
 'registrationDate': '1992-12-14',
 'authorizedCapital': 10000,
 'report_periods': {'2021': 14606812, '2020': 7721040, '2019': 3056434},
 '2021_1110': None,
 '2020_1110': None,
 '2019_1110': None,
 '2021_1120': None,
 '2020_1120': None,
 '2019_1120': None,
 '2021_1130': None,
 '2020_1130': None,
 '2019_1130': None,
 '2021_1140': None,
 '2020_1140': None,
 '2019_1140': None,
 '2021_1150': 0.0,
 '2020_1150': 90.0,
 '2019_1150': 90.0,
 '2021_1160': None,
 '2020_1160': None,
 '2019_1160': None,
 '2021_1170': None,
 '2020_1170': None,
 '2019_1170': None,
 '2021_1180': None,
 '2020_1180': None,
 '2019_1180': None,
 '2021_1190': None,
 '2020_1190': None,
 '2019_1190': None,
 '2021_1100': None,
 '2020_1100': 

- интересно, что по этой ссылке https://bo.nalog.ru/nbo/organizations?page=0 - возвращает json с 20 карточками организации
- в служебной информации jsonа показывает что есть 151683 страниц, но возвращает максимум 500: https://bo.nalog.ru/nbo/organizations?page=499 - по 20 организаций на странице, всего можно забрать 10000, далее - HTTP Status 500 – Internal Server Error

- на данный момент паук обрабатывает кусок инн в 50000 * 4 перехода = около 200 тыс запросов (меньше, так как не для всех инн выдаётся информация), с DOWNLOAD_DELAY = 0.5 и CONCURRENT_REQUESTS = 32 (примерно 100 запросов и 25 записей в минуту) это займёт около 30 часов, выйдёт примерно 40 тыс. записей.
- единичные запросы всё-таки распознаются некорректно (хотя jsonы очень чистые), но причины на первый взгляд не обнаружила, можно было бы добавить дополнительную обработку id и проверку на структуру возвращаемого json-а
- запись идёт в MongoDB (проще и быстрее, чем писать в файл)
- для следующего куска попробую уменьшить задержку (и может быть разбить его помельче) и посмотреть, как будет реагировать сервер.

- остаётся распарсить множество xml
- и решить, стоит ли их куда-нибудь складывать тоже (прямой необходимости, вроде бы, нет), или просто писать в файл.